In [11]:
from zipfile import ZipFile
zip_path = '/content/drive/MyDrive/ColabDrive/IIITB/OneDrive_71_10-30-2024.zip'
extract_dir = '/content/dataset'
with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [12]:
import os
dir_path = '/content/dataset'
files = os.listdir(dir_path)
print(len(files))

71


In [7]:
gpu_info  = !nvidia-smi
gpu_info = "\n".join(gpu_info)
print(gpu_info)

Tue Nov 12 04:47:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
import os
import torch
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [9]:
model_name = "TheAIchemist13/kannada_beekeeping_wav2vec2"
tokenizer = Wav2Vec2Tokenizer.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/854 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/56.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:720: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(


config.json:   0%|          | 0.00/2.23k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at TheAIchemist13/kannada_beekeeping_wav2vec2 were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at TheAIchemist13/kannada_beekeeping_wav2vec2 and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

In [10]:
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder)

In [13]:
transcriptions = {}

In [14]:
for filename in os.listdir(dir_path):
    if filename.endswith(".mp3"):
        audio_file = os.path.join(dir_path, filename)
        audio_input, _ = librosa.load(audio_file, sr=16000)
        input_values = tokenizer(audio_input, return_tensors="pt", padding=True).input_values.to(device)

        max_input_length = 400000
        if input_values.size(1) > max_input_length:
            input_values = input_values[:, :max_input_length]

        try:
            with torch.no_grad():
                logits = model(input_values).logits

            predicted_ids = torch.argmax(logits, dim=-1)
            transcription = tokenizer.batch_decode(predicted_ids)[0]

            transcriptions[filename] = transcription
            print(f"Processed {filename}: {transcription}")

        except RuntimeError as e:
            print(f"Error processing {filename}: {e}. Skipping this file.")

Processed SandalWoodNewsStories_52.mp3: ವಿಡಿಯೋ ಬಳಜ ಗಟೆಗೆ ಸ್ಕರ ಯವೇದ ಪಂಚಕರ್ಮ ಹಾಗೂ ಯೋಗ ವಿಡಿಯಗೊಳಗಾಗಿ ನೋಡುತ್ತ ಇರೆ ನಮ್ಮ ಚಾನಲ್   ಚನಲ್ ಗ ಸಬ್ಸಕ್ರೈಬರಾಗಿ  ವ  ಗಟೆಬಟತ ಲೈಕ್ ಮಾಡಿ ಶೇರ್ ಮಾಡಿ ರಿವಿಕರ ನನು ಡಾ್ಟರ್ ಪಿಕೆ ಪ್ರವೇಡ್ಮಾು ಆಯವೇದ ತ್ತು ಮರ್ಮಚಿಕಿತ್ಸಕ
Processed SandalWoodNewsStories_279.mp3: ೂಕಬರೆ  ಡಿಗಬತ್ತನೆ ಮಲ್ಟಿಲ್ ಫ್ಯಾಕ್ಚರ್ ಆಗುತ್ತೆ ಇಮಿಟ  ಕರಣವ್ು ಮೊಬಲಯತ ಮಿಟ ್ಲೋಕೇಷನ್ ಬರಲಾಗುತ್ತೆ ಕಟವಾಗಬೇಕಾದರೆ ಎಷ್ಟು ದಪ್ಪ ರಬೇಕು ಅಂತ ಪ್ರಶ್ನೆಗೇಳುತ್ತರೆ ಬ್ಲ್ಯಾ್ದೆ ಟೋಟಲ್   ದ ನೋಡತ್ತ ಇದು ಳ್ಳೆ ಸಿಟ    ಲೈಕ್ ಡೈಮಂಡದ್ದಂಗಿದೆಗೆ ಗೋಡ ಡೈಮಾಂಡಿದದೆ ಬೆಳಲ ಪರಮಿಷನ್ ಬೇಕಾಗಿಲ್ಲ ಕಟಮಾಡಿಕ್ಕಬೇಕು
Processed SandalWoodNewsStories_294.mp3: ವರಷದಮೇಲೆ  ಗೌರ್ಮೆಂಟ್ ನೀವು ಸಾಲ ಕೊ ರೀತಿಬಳತೀರಾ ಿಮಗೆ ಸರ ಕೊರಾಟಗರ್ಮೆಂಟ ರ     ಲ್ ಕೂಡ ಇನ್ಸ್ಪರ್ಆಗಿದ್ದೇನೆ  ದು ಕಜಾಗ ದೆ ಜಾಗದಲ್ಲಿ ಸಸಿಗಬೇಕು  ಸಗಳಿಗೆತ್ತ
Processed SandalWoodNewsStories_49.mp3: ರಬಾದಿಗಿಡ  ಹಾ ಿಟು  ಸಿಮಾರ   ನಡ  ಬೆಟ್ಟ ಗು್ಡಗಳ ಮಧ್ಯ ಇದಿಷ್ಟು ನಿ   ಇ ಅಲ್ಲಿರಿ ಕಣದಲ್ಲ ಗಂಧ ಪುರಗದದ  ಕಿಷ್ಣಪರ
Processed SandalWoodNewsStories_89.mp3: ಪಸಾಥಿ ಕೊಟ್ಟಿತ್ತು ದರ    ನ್  ಗೋಬೇಕರ್  ದಿಲ್ಲ      ಬರೇ ೀಗ  ನಿ ನಡೆಯುವುದ ತ್ರು  ಮಧ್ಯಮು ನಾವೇು ಮಾಡಬೇಕದರೆ ್ರೀಗ ಟಟಿತುಂಬ ಟ್ಟಿದತಂಬ 

In [15]:
with open("transcriptions.txt", "w") as f:
    for filename, transcription in transcriptions.items():
        f.write(f"{filename}: {transcription}\n")